# `waterSpec` Demo

This notebook provides a detailed walkthrough of the `waterSpec` package using its primary high-level API, the `Analysis` class. We will see how to perform a full analysis in just a few lines of code, and then explore more advanced options.

## 1. Imports

In [ ]:
import os
import pprint

from IPython.display import Image

from waterSpec import Analysis

## 2. The Main Workflow

The `Analysis` class is the recommended entry point for most users. You initialize it with your data file and column names, and it handles all the preprocessing. Then, you call `run_full_analysis()` to perform the complete workflow and generate outputs.

In [ ]:
# Define paths for our data and where to save outputs
file_path = 'sample_data.csv'
output_dir = 'demo_output'

# 1. Initialize the Analysis object.
# This one-liner loads the data, validates it, and runs all preprocessing steps.
analyzer = Analysis(
    file_path=file_path,
    time_col='timestamp',
    data_col='concentration',
    param_name='Sample Concentration' # Optional: for better plot titles and summaries
)

# 2. Run the full analysis workflow.
# This performs model fitting (auto-selecting standard vs. segmented),
# detects significant peaks, and saves all outputs.
results = analyzer.run_full_analysis(output_dir=output_dir)

print(f"Analysis complete. Outputs saved to '{output_dir}'.")

## 3. Inspecting the Results

The `run_full_analysis` method returns a dictionary containing all results, including the fitted parameters, confidence intervals, and the automated interpretation.

In [ ]:
# The returned dictionary contains all results.
print("--- Full Analysis Results ---")
pprint.pprint(results)

# You can directly access the generated summary text.
print("\n--- Summary Text ---")
print(results['summary_text'])

# You can also see which model (standard or segmented) was chosen automatically based on BIC.
print(f"\nChosen model: {results['chosen_model']}")

## 4. Visualizing the Output

The analysis automatically generates and saves a plot of the spectrum, the fit, and the interpretation summary. We can display it directly in the notebook.

In [ ]:
# Construct the path to the output plot
sanitized_name = analyzer._sanitize_filename(analyzer.param_name)
plot_path = os.path.join(output_dir, f"{sanitized_name}_spectrum_plot.png")

# Display the image
Image(filename=plot_path)

## 5. Advanced Usage

The `Analysis` class constructor accepts several arguments to customize the analysis for different kinds of data.

### 5a. Handling Censored Data

In [ ]:
# Using a dataset with censored values (e.g., '<5')
censored_analyzer = Analysis(
    file_path='censored_data.csv',
    time_col='timestamp',
    data_col='concentration',
    param_name='Censored Concentration',
    censor_strategy='multiplier', # Use the 'multiplier' strategy
    censor_options={'lower_multiplier': 0.5} # Replace '<X' with X * 0.5
)

censored_results = censored_analyzer.run_full_analysis(output_dir=os.path.join(output_dir, 'censored'))

print("--- Censored Data Analysis Summary ---")
print(censored_results['summary_text'])

### 5b. Using Non-Linear Detrending (LOESS)

In [ ]:
# Use the LOESS detrending method instead of the default 'linear'
loess_analyzer = Analysis(
    file_path='sample_data.csv',
    time_col='timestamp',
    data_col='concentration',
    param_name='LOESS Detrended',
    detrend_method='loess'
)

loess_results = loess_analyzer.run_full_analysis(output_dir=os.path.join(output_dir, 'loess'))

print("--- LOESS Detrended Analysis Summary ---")
print(loess_results['summary_text'])

### 5c. Including Measurement Errors

In [ ]:
# Use data with an error column to perform a weighted analysis.
# This gives more weight to more certain data points.
error_analyzer = Analysis(
    file_path='sample_data_with_errors.csv',
    time_col='timestamp',
    data_col='concentration',
    error_col='error', # Specify the error column
    param_name='Weighted Analysis'
)

error_results = error_analyzer.run_full_analysis(output_dir=os.path.join(output_dir, 'weighted'))

print("--- Weighted Analysis Summary ---")
print(error_results['summary_text'])